# 任意のクエリでTF-IDF求める

In [10]:
import copy
import functools as ft
import json
import os
import random as rd

from gensim import corpora
from gensim import models
from operator import itemgetter
from operator import itemgetter

import controll_text as ct

In [17]:
files = os.listdir('mayfes/json_data/2ndLayer/')
querys = []
for file in files:
    query,_ = os.path.splitext(file)
    if query.startswith('.'):
        pass
    else:
        querys.append(query)
print(len(querys))

508


In [19]:
picked_number = 10
while len(querys) > 0:
    picked_number = min(picked_number,len(querys))
    random_querys = rd.sample(querys,picked_number)
    for query in random_querys:
        querys.remove(query)
    print('今回のクエリ: ' + str(random_querys))
    print('残りクエリ数: ' + str(len(querys)))


    query_texts = dict()
    for query in random_querys:
        row_data = open('mayfes/json_data/2ndLayer/{0}.json'.format(query), 'r')
        data = json.load(row_data)
        text_list = list(map(lambda d: ft.reduce(lambda l1, l2: l1+l2, d["texts"]), filter(lambda d: d["texts"] !=[], data)))
        query_texts[query] = text_list


    noun_conditions = ct.make_noun_conditions(except_nouns=['/', '(', ')', '>', '<', '-', '.','"','_', 'ー','＿','ー',';',':','·','...'])
    verb_conditions = ct.make_verb_conditions(except_verbs=[], verb_types = [])
    query_words = {}
    for k, v in query_texts.items():
        query_words[k] = list(map(ft.partial(ct.extract_words, noun_conditions,verb_conditions), v))
    # query_words["サイト"][0:1]



    def tfidf(full_texts, target_words_list):
        dictionary = corpora.Dictionary(full_texts)
        n_words = len(dictionary.token2id)
        id2token = {v: k for k, v in dictionary.token2id.items()}
        full_corpus = map(dictionary.doc2bow, full_texts)
        target_corpus = map(dictionary.doc2bow, target_words_list)
        tfidf_model = models.TfidfModel(copy.deepcopy(full_corpus))
        tfidf_vectors_by_tuple = list(tfidf_model[copy.deepcopy(target_corpus)])

        # TF-IDF行列を算出
        vectors = []
        for v_by_tuple in tfidf_vectors_by_tuple:
            vector = [0]*n_words 
            for t in v_by_tuple:
                vector[t[0]] = t[1]
            vectors.append(vector)
        # 行列を転置
        vectors_t = list(zip(*vectors))

        #単語ごとにTF-IDFベクトルの平均値を求める
        vectors_t2 = [list(it) for it in list(zip(*vectors))]
        ave_vec = list(map(lambda l: sum(l)/len(l), vectors_t2))
        tfidf_ave_dict = {i:v for i, v in enumerate(ave_vec)}

        word_dict = dict()
        for k,v in tfidf_ave_dict.items():
            word_dict[id2token[k]] = v

        return sorted(word_dict.items(), key=itemgetter(1), reverse=True)

    # 全文
    full_texts = ft.reduce(lambda l1, l2:l1+l2, query_words.values())


    # 出力
    for query in random_querys:
        target_words_list = query_words[query]
        ranking = tfidf(full_texts, target_words_list)
        f = open('./tfidf_text/2ndLayer/{0}.txt'.format(query), 'w')
        for x in ranking:
            f.write(str(x) + "\n")
        f.close()

今回のクエリ: ['プログラミング', '映画', '予算', '地震', '募集', '中国', 'ID', '本願寺', 'ローン', '一戸建て']
残りクエリ数: 498
今回のクエリ: ['番組', '満載', '透析', '千葉', 'デザイン', '分譲', '履修', '連合', 'http', '画像']
残りクエリ数: 488
今回のクエリ: ['カリキュラム', '夢', 'インターネット', '専門', 'カタログ', '休', '福祉', '受験', '全国', '気象']
残りクエリ数: 478
今回のクエリ: ['性', '公立', '掲載', '理解', '各種', '幕張', 'テーマ', '配送', '目黒', '月行事']
残りクエリ数: 468
今回のクエリ: ['日吉', '〜', '居酒屋', 'バー', '一覧', '駅', 'センター', '急性', '推薦', 'アマゾン']
残りクエリ数: 458
今回のクエリ: ['ゲーム', '個人', '医学', '年間', 'アットホーム', '免許', '方法', '病気', '委員', '本文']
残りクエリ数: 448
今回のクエリ: ['申込', '新橋', '宿泊', '人間', '国会図書館', '附属', 'アリーナ', 'ニュース', '駐車', 'PS']
残りクエリ数: 438
今回のクエリ: ['高知', '期間', '支払い', 'AO', 'ターミナル', '挨拶', '製品', '出演', '治療', '計画']
残りクエリ数: 428
今回のクエリ: ['体験', 'PlayStation', 'フライト', '位置', '観光', 'ぞう', 'セガ', '神奈川', '単位', '分野']
残りクエリ数: 418
今回のクエリ: ['工業', '庁舎', '教室', 'ビジョン', 'ご覧', '特集', 'ページ', '木', '証明', 'アリ']
残りクエリ数: 408
今回のクエリ: ['出願', 'システム', '会議', 'ビジネス', '邑楽', '表情', '質問', '帝京大学', '専攻', '提供']
残りクエリ数: 398
今回のクエリ: ['ボランティア', '郁', '商品', 'プロジェクト', '水', '胃